In [9]:
import numpy as np
import math
from tabulate import tabulate

def ncr(n, r):
    f = math.factorial
    return f(n) // f(r) // f(n-r)

$n^2 +3n + 2 - 2m = 0$

$n = - \frac{3}{2} \pm \sqrt{\frac{1}{4} + 2m}$

In [237]:
def gauss(n):
    return int(n * (n+1) * 0.5)

def getIndex(col, row):
    return gauss(col + row) + col

def getMaxDegree(config):
    return int(-1.5 + np.sqrt(0.25 + 2*config.size))

def printConfig(config):
    max_deg = getMaxDegree(config)
    for len_row, row_index in enumerate(range(max_deg, -1, -1)):
        for col_index in range(len_row + 1):
            val = str(config[getIndex(col_index, row_index)])
            print(f"{''.join([' '] * (4 - len(val))) + val}", end=" ")
        print("\n", end="")
    print("")


def create_exhaust_generator(degree, support_size = 4):
    N = int((degree+1)*(degree+2)/2)
    for d in range(degree + 1):
        diag = getIndex(d, degree - d)
        for i in range(1, diag):
            for j in range(1, i):
                for k in range(1, j):
                    if diag != i and diag != j and diag != k:
                        yield (diag, i, j, k)

In [219]:
def hyperfield_add(A, B):
    res = np.sign(A + B)
    res[(A == -B) & (A != 0)] = np.nan
    res[np.isnan(A) | np.isnan(B)] = np.nan
    return res

def eval_f(support,x):
    support_pos, support_neg = support[0], support[1]
    if any(np.isnan(x[support_pos | support_neg])):
        return np.nan
    has_neg = any(x[support_pos] < 0) | any(x[support_neg] > 0)
    has_pos = any(x[support_pos] > 0) | any(x[support_neg] < 0)
    if has_neg and has_pos:
        return np.nan
    if has_neg:
        return -1
    elif has_pos:
        return 1
    return 0

### Lemma 6.9
$\mathrm{sign}(\varphi_{a,b})(x) = \sum^{a}_{i=0} \sum^{b}_{j=0} x_{i,j}$

In [227]:
def varphi(degree, a):
    support_pos = np.full(gauss(degree + 1), False)
    support_neg = np.full(gauss(degree + 1), False)
    for x in range(a+1):
        for y in range(degree - a, -1, -1):
            support_pos[getIndex(x,y)] = True
    return support_pos, support_neg

def psi(degree, k):
    support_pos = np.full(gauss(degree + 1), False)
    support_neg = np.full(gauss(degree + 1), False)
    for j in range(k+1):
        for i in range(k-j, degree-j + 1):
            if (k - j) % 2 == 0:
                support_pos[getIndex(i,j)] = True
            else:
                support_neg[getIndex(i,j)] = True
    return support_pos, support_neg

def psi_bar(degree, k):
    support_pos = np.full(gauss(degree + 1), False)
    support_neg = np.full(gauss(degree + 1), False)
    for j in range(k+1):
        for i in range(k-j, degree-j + 1):
            if (k - j) % 2 == 0:
                support_pos[getIndex(j,i)] = True
            else:
                support_neg[getIndex(j,i)] = True
    return support_pos, support_neg

In [95]:
config = np.array([0,1,2,3,4,5,6,7,8,9], dtype="int")
printConfig(config)

| \
|6 \
|3 7 \
|1 4 8 \
|0 2 5 9 \
- - - - - - 


In [117]:
gen = create_exhaust_generator(5)
print(len(list(gen)))

3844


## Count valid configurations

Goal: count the number of configurations $s \in H^{V_d}$ that are valid and have degree $deg \in \{2,3,..., 7 \}$.

In [118]:
table = [['support size', 'degree', 'total']]

support_size = 4
for deg in range(2, 8):
    count_diag = deg + 1
    count_lower_triangle = gauss(deg) - 1

    sum = 0
    # we start with 1 since every configuration should have degree deg
    for k in range(1, support_size +1):
        if count_diag >= k and count_lower_triangle >= support_size - k:
            sum += ncr(count_diag, k) * ncr(count_lower_triangle, support_size - k)
    table.append([support_size, deg, sum])

print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒════════════════╤══════════╤═════════╕
│   support size │   degree │   total │
╞════════════════╪══════════╪═════════╡
│              4 │        2 │       5 │
├────────────────┼──────────┼─────────┤
│              4 │        3 │     121 │
├────────────────┼──────────┼─────────┤
│              4 │        4 │     875 │
├────────────────┼──────────┼─────────┤
│              4 │        5 │    3844 │
├────────────────┼──────────┼─────────┤
│              4 │        6 │   12705 │
├────────────────┼──────────┼─────────┤
│              4 │        7 │   34810 │
╘════════════════╧══════════╧═════════╛


### Example deg = 2, support size = 4
Here we enumerate all valid configurations of degree $2$.

In [109]:
degree = 2
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    printConfig(w)

| \
|1 \
|1 1 \
|0 1 0 \
- - - - 
| \
|1 \
|1 0 \
|0 1 1 \
- - - - 
| \
|0 \
|1 1 \
|0 1 1 \
- - - - 
| \
|1 \
|1 1 \
|0 0 1 \
- - - - 
| \
|1 \
|0 1 \
|0 1 1 \
- - - - 


### Example deg = 3, support size = 4
Here we enumerate all valid configurations of degree $3$.

In [103]:
degree = 3
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    printConfig(w)

| \
|1 \
|1 0 \
|1 0 0 \
|0 1 0 0 \
- - - - - - 
| \
|1 \
|0 0 \
|1 1 0 \
|0 1 0 0 \
- - - - - - 
| \
|1 \
|1 0 \
|1 1 0 \
|0 0 0 0 \
- - - - - - 
| \
|1 \
|1 0 \
|0 1 0 \
|0 1 0 0 \
- - - - - - 
| \
|1 \
|0 0 \
|1 0 0 \
|0 1 1 0 \
- - - - - - 
| \
|1 \
|1 0 \
|1 0 0 \
|0 0 1 0 \
- - - - - - 
| \
|1 \
|1 0 \
|0 0 0 \
|0 1 1 0 \
- - - - - - 
| \
|1 \
|0 0 \
|1 1 0 \
|0 0 1 0 \
- - - - - - 
| \
|1 \
|0 0 \
|0 1 0 \
|0 1 1 0 \
- - - - - - 
| \
|1 \
|1 0 \
|0 1 0 \
|0 0 1 0 \
- - - - - - 
| \
|0 \
|1 1 \
|1 0 0 \
|0 1 0 0 \
- - - - - - 
| \
|0 \
|0 1 \
|1 1 0 \
|0 1 0 0 \
- - - - - - 
| \
|0 \
|1 1 \
|1 1 0 \
|0 0 0 0 \
- - - - - - 
| \
|0 \
|1 1 \
|0 1 0 \
|0 1 0 0 \
- - - - - - 
| \
|0 \
|0 1 \
|1 0 0 \
|0 1 1 0 \
- - - - - - 
| \
|0 \
|1 1 \
|1 0 0 \
|0 0 1 0 \
- - - - - - 
| \
|0 \
|1 1 \
|0 0 0 \
|0 1 1 0 \
- - - - - - 
| \
|0 \
|0 1 \
|1 1 0 \
|0 0 1 0 \
- - - - - - 
| \
|0 \
|0 1 \
|0 1 0 \
|0 1 1 0 \
- - - - - - 
| \
|0 \
|1 1 \
|0 1 0 \
|0 0 1 0 \
- - - - - - 
| \
|1 \
|0 1 \
|1 0

### Proof of Lemma 6.21 a,b,c

Goal: Find the number of valid configurations $s \in H^{V_d}$ with $|\mathrm{supp}^+(s)| = 4$ such that all basis systems $\varphi, \psi, \bar \psi$ vanish at $s$.

In [252]:
%%time
degree = 6
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")

Number of configurations: 5
CPU times: user 692 ms, sys: 3.24 ms, total: 695 ms
Wall time: 698 ms


In [254]:
%%time
degree = 7
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")

Number of configurations: 3
CPU times: user 2 s, sys: 2.23 ms, total: 2 s
Wall time: 2.01 s


In [256]:
%%time
degree = 8
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")

Number of configurations: 0
CPU times: user 5.2 s, sys: 3.33 ms, total: 5.2 s
Wall time: 5.23 s


In [259]:
%%time
degree = 9
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")

Number of configurations: 0
CPU times: user 12.2 s, sys: 4.89 ms, total: 12.2 s
Wall time: 12.3 s


In [260]:
%%time
degree = 10
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")

Number of configurations: 0
CPU times: user 26.9 s, sys: 10 ms, total: 26.9 s
Wall time: 27.1 s


In [261]:
%%time
degree = 11
res = []
basis = [varphi(degree, a) for a in range(degree + 1)]
basis2 = [psi(degree, k) for k in range(degree + 1)]
basis3 = [psi_bar(degree, k) for k in range(degree + 1)]
for supp in create_exhaust_generator(degree):
    w = np.array([1 if idx in supp else 0 for idx in range(gauss(degree+1))])
    w[0] = -1
    if np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis2))))) and np.all(np.isnan(np.array(list(map(lambda f: eval_f(f, w), basis3))))):
        res.append(w)
print(f"Number of configurations: {len(res)}")

Number of configurations: 0
CPU times: user 55.4 s, sys: 9.02 ms, total: 55.4 s
Wall time: 55.6 s



__Conclusion__: We conclude that only configurations $s$ of degree $d = 6$ or $d = 7$ belong to $\Omega_d$.